In [11]:
import pygame
import math

# Initialize Pygame
pygame.init()

# Constants
WINDOW_SIZE = (600, 600)  # Larger window to fit the triangle
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
FONT_COLOR = (0, 0, 0)
FONT_SIZE = 24
step_size = 10
tolerance = 10  # How close the agent needs to be to a vertex to trigger the end

# Set up display
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("Triangle Environment")

# Load font for rendering text
font = pygame.font.Font(None, FONT_SIZE)

# Environment variables
# Define the vertices of an equilateral triangle
triangle_vertices = [
    (300, 100),  # Top vertex
    (100, 500),  # Bottom-left vertex
    (500, 500)   # Bottom-right vertex
]

# Agent starts in the middle of the triangle
agent_pos = [300, 300]  # The center of the triangle
reached_goal = False  # Track if a goal (vertex) has been reached

# Function to calculate if the point is inside the triangle
def point_in_triangle(p, v1, v2, v3):
    def sign(p1, p2, p3):
        return (p1[0] - p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])

    d1 = sign(p, v1, v2)
    d2 = sign(p, v2, v3)
    d3 = sign(p, v3, v1)
    has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
    has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)
    return not (has_neg and has_pos)

# Function to check if agent reached a vertex
def check_vertex_reached():
    global reached_goal
    for vertex in triangle_vertices:
        if distance(agent_pos, vertex) < tolerance:
            reached_goal = True
            break

# Function to calculate distance between two points
def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Function to move the agent in a specific direction
def move_agent(direction):
    global agent_pos
    # Move based on direction
    if direction == "UP":
        new_pos = [agent_pos[0], agent_pos[1] - step_size]
    elif direction == "DOWN":
        new_pos = [agent_pos[0], agent_pos[1] + step_size]
    elif direction == "LEFT":
        new_pos = [agent_pos[0] - step_size, agent_pos[1]]
    elif direction == "RIGHT":
        new_pos = [agent_pos[0] + step_size, agent_pos[1]]

    # Check if new position is within the triangle
    if point_in_triangle(new_pos, triangle_vertices[0], triangle_vertices[1], triangle_vertices[2]):
        agent_pos = new_pos  # Update agent position if within triangle bounds

    # Check if agent has reached a vertex
    check_vertex_reached()

# Function to reset the environment
def reset():
    global agent_pos, reached_goal
    agent_pos = [300, 300]  # Reset agent to the center
    reached_goal = False  # Reset goal flag

# Function to render the environment and GUI
def render():
    # Clear the screen
    screen.fill(WHITE)

    # Draw the triangle
    pygame.draw.polygon(screen, BLUE, triangle_vertices, 2)

    # Draw agent (green circle)
    pygame.draw.circle(screen, GREEN, (int(agent_pos[0]), int(agent_pos[1])), 10)

    # Draw vertices (red circles)
    for vertex in triangle_vertices:
        pygame.draw.circle(screen, RED, vertex, 10)

    # If the goal is reached, display a message
    if reached_goal:
        text = font.render("Goal Reached! Press R to reset", True, FONT_COLOR)
        screen.blit(text, (150, 50))

    # Update the display
    pygame.display.update()

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:  # Reset the game if 'R' is pressed
                reset()

    # Check if the agent has reached a vertex
    if not reached_goal:  # Only allow movement if goal hasn't been reached
        # Get keys pressed
        keys = pygame.key.get_pressed()

        # Move based on keys
        if keys[pygame.K_UP]:        # Move strictly up
            move_agent("UP")
        if keys[pygame.K_DOWN]:      # Move strictly down
            move_agent("DOWN")
        if keys[pygame.K_LEFT]:      # Move strictly left
            move_agent("LEFT")
        if keys[pygame.K_RIGHT]:     # Move strictly right
            move_agent("RIGHT")

    # Render the environment
    render()

    pygame.time.wait(100)  # Slow down for visibility

# Quit Pygame
pygame.quit()
